In [83]:
import pandas as pd
import requests
import math
from decimal import Decimal

In [84]:

symbol = "BTCUSDT"
url = "https://api.binance.com/api/v3/depth"



params = {
    "symbol":symbol,
    "limit":5000,
}

In [85]:
data = requests.get(url, params).json()
data

{'lastUpdateId': 27220557459,
 'bids': [['16981.17000000', '0.03658000'],
  ['16978.48000000', '0.01110000'],
  ['16978.47000000', '0.06666000'],
  ['16978.46000000', '0.03768000'],
  ['16978.45000000', '0.26597000'],
  ['16978.41000000', '0.04992000'],
  ['16978.24000000', '0.05889000'],
  ['16978.05000000', '0.00675000'],
  ['16978.00000000', '0.00092000'],
  ['16977.51000000', '0.00800000'],
  ['16977.30000000', '0.35341000'],
  ['16977.11000000', '0.29828000'],
  ['16977.02000000', '0.18600000'],
  ['16977.00000000', '0.01595000'],
  ['16976.92000000', '0.03105000'],
  ['16976.79000000', '0.19806000'],
  ['16976.53000000', '0.14046000'],
  ['16976.48000000', '0.00100000'],
  ['16976.35000000', '0.25299000'],
  ['16976.15000000', '0.00993000'],
  ['16976.10000000', '0.06480000'],
  ['16976.00000000', '0.00092000'],
  ['16975.36000000', '0.45201000'],
  ['16975.35000000', '0.04000000'],
  ['16975.32000000', '0.09661000'],
  ['16975.15000000', '0.50431000'],
  ['16974.95000000', '0.70

In [86]:
factor_price = data["bids"][0][0]
precision_price = len(factor_price.split(".")[-1])
factor_price = 10**precision_price

interval = Decimal('50')

interval = int(interval * factor_price)

In [87]:
factor_quant = data["bids"][0][1]
precision_quant = len(factor_quant.split(".")[-1])
factor_quant = 10**precision_quant

In [88]:
bids = data["bids"]
bids = [ [ int(Decimal(x[0])*factor_price), int(Decimal(x[1])*factor_quant)] for x in bids]

asks = data["asks"]
asks = [ [ int(Decimal(x[0])*factor_price), int(Decimal(x[1])*factor_quant)] for x in asks]

asks

[[1698132000000, 2999000],
 [1698166000000, 1392000],
 [1698209000000, 177000],
 [1698217000000, 6997000],
 [1698218000000, 11794000],
 [1698235000000, 5700000],
 [1698258000000, 329000],
 [1698260000000, 304000],
 [1698261000000, 659000],
 [1698305000000, 103000000],
 [1698330000000, 24510000],
 [1698331000000, 13350000],
 [1698355000000, 2398000],
 [1698358000000, 2999000],
 [1698361000000, 32000000],
 [1698370000000, 8091000],
 [1698380000000, 2000000],
 [1698390000000, 1499000],
 [1698400000000, 92000],
 [1698412000000, 665000],
 [1698426000000, 29828000],
 [1698444000000, 457000],
 [1698445000000, 22197000],
 [1698446000000, 71000],
 [1698474000000, 999000],
 [1698477000000, 17679000],
 [1698490000000, 5000000],
 [1698494000000, 11679000],
 [1698495000000, 62042000],
 [1698496000000, 5000000],
 [1698506000000, 5000000],
 [1698507000000, 5100000],
 [1698517000000, 294000],
 [1698518000000, 394000],
 [1698535000000, 14046000],
 [1698562000000, 321000],
 [1698613000000, 999000],
 [16

In [89]:
bid_levels = pd.DataFrame(bids, columns = ['price','quantity'], dtype = int)
bid_levels["side"] = "bid"

print(bid_levels)

min_bid_level = math.floor(min(bid_levels.price) / interval)*interval
max_bid_level = (math.ceil(max(bid_levels.price) / interval) + 1)*interval

bid_level_bounds = [ min_bid_level + interval*x for x in 
                       range( int((max_bid_level - min_bid_level) / interval) + 1 )]

bid_levels["bin"] = pd.cut(bid_levels.price, bins=bid_level_bounds, right = False,
                          precision = 10)

bid_levels = bid_levels.groupby("bin").agg(
        quantity = ("quantity", "sum"),
        side = ("side", "first")).reset_index()

bid_levels["label"] = bid_levels.bin.apply(lambda x: x.left)

bid_levels

              price  quantity side
0     1698117000000   3658000  bid
1     1697848000000   1110000  bid
2     1697847000000   6666000  bid
3     1697846000000   3768000  bid
4     1697845000000  26597000  bid
...             ...       ...  ...
4995  1642626000000    506000  bid
4996  1642600000000   1704000  bid
4997  1642581000000     69000  bid
4998  1642568000000    103000  bid
4999  1642552000000    301000  bid

[5000 rows x 3 columns]


,bin,quantity,side,label
0,"[1640000000000, 1645000000000)",750853000,bid,1640000000000
1,"[1645000000000, 1650000000000)",4099383000,bid,1645000000000
2,"[1650000000000, 1655000000000)",50954164300,bid,1650000000000
3,"[1655000000000, 1660000000000)",7418317000,bid,1655000000000
4,"[1660000000000, 1665000000000)",9338676600,bid,1660000000000
5,"[1665000000000, 1670000000000)",18598574000,bid,1665000000000
6,"[1670000000000, 1675000000000)",7354173200,bid,1670000000000
7,"[1675000000000, 1680000000000)",7497474800,bid,1675000000000
8,"[1680000000000, 1685000000000)",32223650700,bid,1680000000000
9,"[1685000000000, 1690000000000)",34899845000,bid,1685000000000


In [90]:
ask_levels = pd.DataFrame(asks, columns = ['price','quantity'], dtype = int)
ask_levels["side"] = "ask"

min_ask_level = (math.floor(min(ask_levels.price) / interval) - 1)*interval
max_ask_level = (math.ceil(max(ask_levels.price) / interval) + 1)*interval

ask_level_bounds = [ min_ask_level + interval*x for x in 
                       range( int((max_ask_level - min_ask_level) / interval) + 1 )]

ask_levels["bin"] = pd.cut(ask_levels.price, bins=ask_level_bounds, right = True,
                          precision = 10)

ask_levels = ask_levels.groupby("bin").agg(
        quantity = ("quantity", "sum"),
        side = ("side", "first")).reset_index()

ask_levels["label"] = ask_levels.bin.apply(lambda x: x.right)

ask_levels

,bin,quantity,side,label
0,"(1690000000000, 1695000000000]",0,None,1695000000000
1,"(1695000000000, 1700000000000]",2104511000,ask,1700000000000
2,"(1700000000000, 1705000000000]",20220099000,ask,1705000000000
3,"(1705000000000, 1710000000000]",26281808000,ask,1710000000000
4,"(1710000000000, 1715000000000]",6000577000,ask,1715000000000
5,"(1715000000000, 1720000000000]",5376599000,ask,1720000000000
6,"(1720000000000, 1725000000000]",7468521000,ask,1725000000000
7,"(1725000000000, 1730000000000]",11924446000,ask,1730000000000
8,"(1730000000000, 1735000000000]",2058849000,ask,1735000000000
9,"(1735000000000, 1740000000000]",4628651000,ask,1740000000000


In [91]:
orderbook = pd.concat([ask_levels, bid_levels])

orderbook = orderbook[ orderbook.quantity > 0]

print(orderbook.sort_values("label", ascending = False).to_string())

                               bin     quantity side          label
14  (1760000000000, 1765000000000]   1071610000  ask  1765000000000
13  (1755000000000, 1760000000000]   3162439000  ask  1760000000000
12  (1750000000000, 1755000000000]   1490373000  ask  1755000000000
11  (1745000000000, 1750000000000]   5345513000  ask  1750000000000
10  (1740000000000, 1745000000000]   6993777000  ask  1745000000000
9   (1735000000000, 1740000000000]   4628651000  ask  1740000000000
8   (1730000000000, 1735000000000]   2058849000  ask  1735000000000
7   (1725000000000, 1730000000000]  11924446000  ask  1730000000000
6   (1720000000000, 1725000000000]   7468521000  ask  1725000000000
5   (1715000000000, 1720000000000]   5376599000  ask  1720000000000
4   (1710000000000, 1715000000000]   6000577000  ask  1715000000000
3   (1705000000000, 1710000000000]  26281808000  ask  1710000000000
2   (1700000000000, 1705000000000]  20220099000  ask  1705000000000
1   (1695000000000, 1700000000000]   2104511000 

In [92]:
orderbook.quantity /= factor_quant
orderbook.label /= factor_price

print(orderbook[["quantity", "side", "label"]].sort_values("label", ascending=False).to_string())

      quantity side    label
14   10.716100  ask  17650.0
13   31.624390  ask  17600.0
12   14.903730  ask  17550.0
11   53.455130  ask  17500.0
10   69.937770  ask  17450.0
9    46.286510  ask  17400.0
8    20.588490  ask  17350.0
7   119.244460  ask  17300.0
6    74.685210  ask  17250.0
5    53.765990  ask  17200.0
4    60.005770  ask  17150.0
3   262.818080  ask  17100.0
2   202.200990  ask  17050.0
1    21.045110  ask  17000.0
11   64.838970  bid  16950.0
10  272.618700  bid  16900.0
9   348.998450  bid  16850.0
8   322.236507  bid  16800.0
7    74.974748  bid  16750.0
6    73.541732  bid  16700.0
5   185.985740  bid  16650.0
4    93.386766  bid  16600.0
3    74.183170  bid  16550.0
2   509.541643  bid  16500.0
1    40.993830  bid  16450.0
0     7.508530  bid  16400.0


In [93]:
print(orderbook[["quantity", "side", "label"]].sort_values("label", ascending=False).to_string())

      quantity side    label
14   10.716100  ask  17650.0
13   31.624390  ask  17600.0
12   14.903730  ask  17550.0
11   53.455130  ask  17500.0
10   69.937770  ask  17450.0
9    46.286510  ask  17400.0
8    20.588490  ask  17350.0
7   119.244460  ask  17300.0
6    74.685210  ask  17250.0
5    53.765990  ask  17200.0
4    60.005770  ask  17150.0
3   262.818080  ask  17100.0
2   202.200990  ask  17050.0
1    21.045110  ask  17000.0
11   64.838970  bid  16950.0
10  272.618700  bid  16900.0
9   348.998450  bid  16850.0
8   322.236507  bid  16800.0
7    74.974748  bid  16750.0
6    73.541732  bid  16700.0
5   185.985740  bid  16650.0
4    93.386766  bid  16600.0
3    74.183170  bid  16550.0
2   509.541643  bid  16500.0
1    40.993830  bid  16450.0
0     7.508530  bid  16400.0
